In [1]:
import pandas as pd
import numpy as np
from scipy.integrate import trapz
from scipy.signal import find_peaks
from lets_plot import *
import pingouin as pg

LetsPlot.setup_html()

# sample information

In [2]:
# importing dataframe
df = pd.read_csv("https://media.githubusercontent.com/media/bvn3141/beta_ala_study/main/Beta_Ala_Dataframe.csv?token=BGUXDREHXAF4WRJIAVB4FG3F6AFJ4",
                sep=';')
df.drop('Unnamed: 7', axis=1, inplace=True)
df

,id,group,age,sex,weight_kg,hight_m,bmi
0,25778.0,Beta Ala,25.0,w,57.0,1.63,21.45
1,54145.0,Beta Ala,21.0,w,72.0,1.72,24.34
2,15073.0,Beta Ala,24.0,m,69.0,1.82,20.38
3,20438.0,Beta Ala,25.0,m,76.0,1.80,23.46
4,49830.0,Beta Ala,26.0,w,82.0,1.68,29.05
5,58046.0,Beta Ala,23.0,m,77.0,1.85,22.50
6,26107.0,Beta Ala,22.0,w,65.0,1.70,22.49
7,44858.0,Beta Ala,26.0,w,53.0,1.64,19.71
8,59289.0,Beta Ala,21.0,m,75.0,1.78,23.67
9,93672.0,Beta Ala,24.0,m,82.0,1.77,26.17


In [3]:
# number of male and female participants
df['sex'].value_counts()

sex
w    12
m    12
Name: count, dtype: int64

In [4]:
# sample by group
frame_by_group = df.groupby('group')
group_statistics = frame_by_group[['age', 'weight_kg', 'hight_m', 'bmi']].agg(['mean', 'std', 'min', 'max'])

pd.set_option('display.float_format', lambda x: '%.2f' % x)

group_statistics = group_statistics.transpose()
group_statistics.index.name = 'Variable'

print('Descriptive Statistics for Each Group\n')
print(group_statistics.to_string())

Descriptive Statistics for Each Group

group           Beta Ala  Placebo
age       mean     24.25    25.58
          std       2.49     2.71
          min      21.00    22.00
          max      30.00    30.00
weight_kg mean     71.83    68.67
          std      11.02     9.92
          min      53.00    58.00
          max      91.00    87.00
hight_m   mean      1.75     1.72
          std       0.10     0.09
          min       1.62     1.54
          max       1.96     1.87
bmi       mean     23.41    22.97
          std       2.51     1.52
          min      19.71    21.05
          max      29.05    25.47


## Import isokinetic raw data from github repository

In [6]:
iso = pd.read_csv("https://media.githubusercontent.com/media/bvn3141/beta_ala_study/main/isokinet_dataframe.csv?token=BGUXDRHQNPSKRSRKXH23DRTF6AJK6")

# format data
iso['id'] = iso['id'].astype('int64')
iso['measurement'] = iso['measurement'].astype('category')
iso['experiment'] = iso['experiment'].astype('category')
iso['group'] = iso['group'].astype('category')

# tranform data to its units
iso['pos_1/10_deg/s_or_mm'] = iso['pos_1/10_deg/s_or_mm'] / 10
iso['torque_1/10nm_or_force_n'] = iso['torque_1/10nm_or_force_n'] / (-10)
iso['speed_1/10_deg/sec_or_1/10_mm/s'] = iso['speed_1/10_deg/sec_or_1/10_mm/s'] / 10


iso.rename(columns={'pos_1/10_deg/s_or_mm': 'pos_deg_or_mm', 'torque_1/10nm_or_force_n': 'torque_nm',
                   'speed_1/10_deg/sec_or_1/10_mm/s': 'speed_deg/s_or_mm/s'}, inplace=True)
iso.head()

,id,measurement,time_ms,pos_deg_or_mm,torque_nm,speed_deg/s_or_mm/s,torque_without_comp,rep,set,torque_dyn,force_right,force_left,experiment,group
0,15073,warmup,5,12.00,-3.00,0.00,293,1,1,30,0,0,pretest,beta_ala
1,15073,warmup,10,12.00,-3.00,0.50,293,1,1,30,0,0,pretest,beta_ala
2,15073,warmup,15,12.00,-2.30,4.00,293,1,1,23,0,0,pretest,beta_ala
3,15073,warmup,20,12.10,-0.00,18.30,293,1,1,0,0,0,pretest,beta_ala
4,15073,warmup,25,12.10,11.30,23.50,293,1,1,-113,0,0,pretest,beta_ala


## Calculate Peak Torque for each participant in pre- and post-test
- peak torque maximum torque acomplished
- colum abbruchkriterium = 50% of maximal torque value. At three consequtive repetitions below that value, sufficient fatigue was induced and the protocol was terminated. pre = pretest, post = posttest

In [7]:
# create dataframe with torque values
pt_results = pd.DataFrame(columns = ['id', 'group', 'peak_torque_pre', 'abbruchkriterium_pre', 'peak_torque_post',
                                     'abbruchkriterium_post'])

probanden_ids = iso['id'].unique()

for proband_id in probanden_ids:
    proband_data = iso[iso['id'] == proband_id]
    
    # add group information
    group = proband_data['group'].iloc[0]
    
    # filter for pre and post measurements
    proband_data_pre = proband_data[proband_data['experiment'] == 'pretest']
    proband_data_post = proband_data[proband_data['experiment'] == 'posttest']
    
    # filter for concentric reps. every second rep
    kon_pre = proband_data_pre.query('measurement == "pt" and rep % 2 == 0')
    kon_post = proband_data_post.query('measurement == "pt" and rep % 2 == 0')
    
    # calculate termination criterion
    # pretest
    
    peak_torque_pre = kon_pre['torque_nm'].max()
    abort_pre = peak_torque_pre / 2
    
    # posttest
    
    peak_torque_post = kon_post['torque_nm'].max()

    # termination at 50% max torque
    abort_post = peak_torque_post / 2
    
    ergebnis_row = pd.DataFrame({
        'id': [proband_id],
        'group': [group],
        'peak_torque_pre': [peak_torque_pre],
        'abbruchkriterium_pre': [abort_pre],
        'peak_torque_post': [peak_torque_post],
        'abbruchkriterium_post': [abort_post]
        
    })
    
    # add results to dataframe
    pt_results = pd.concat([pt_results, ergebnis_row], ignore_index = True)
    
pt_results

C:\Users\bvenn\AppData\Local\Temp\ipykernel_11308\2043861747.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pt_results = pd.concat([pt_results, ergebnis_row], ignore_index = True)


,id,group,peak_torque_pre,abbruchkriterium_pre,peak_torque_post,abbruchkriterium_post
0,15073,beta_ala,249.50,124.75,269.00,134.50
1,20438,beta_ala,306.80,153.40,291.00,145.50
2,25778,beta_ala,163.20,81.60,155.70,77.85
3,26107,beta_ala,201.50,100.75,194.00,97.00
4,27351,beta_ala,162.80,81.40,166.50,83.25
5,28514,placebo,121.20,60.60,139.20,69.60
6,93672,beta_ala,294.50,147.25,338.30,169.15
7,33468,placebo,156.50,78.25,171.00,85.50
8,39337,placebo,296.70,148.35,261.50,130.75
9,39750,placebo,139.20,69.60,131.30,65.65


In [8]:
pt_results_l = pd.melt(pt_results.drop(columns=['abbruchkriterium_pre', 'abbruchkriterium_post']), id_vars = ['id', 'group'], value_name='torque', var_name = 'measurement')
pt_results_l.head()

,id,group,measurement,torque
0,15073,beta_ala,peak_torque_pre,249.50
1,20438,beta_ala,peak_torque_pre,306.80
2,25778,beta_ala,peak_torque_pre,163.20
3,26107,beta_ala,peak_torque_pre,201.50
4,27351,beta_ala,peak_torque_pre,162.80


In [9]:
# Peak Torque group differences

(
    ggplot(pt_results_l, aes(x='measurement', y='torque'))
    + geom_boxplot(aes(group='group', color='group'))
    + labs(title='Peak Torque at pre and post test - seperated by groups.')
    + ylab('Peak Torque [Nm]')
)

In [10]:
pt_stat = pg.mixed_anova(dv='torque',between='group', within='measurement', subject='id', data=pt_results_l, correction='auto', effsize='np2')
pt_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,13313.34,1,22,13313.34,1.68,0.21,0.07,NaN
1,measurement,189.61,1,22,189.61,1.12,0.30,0.05,1.00
2,Interaction,23.52,1,22,23.52,0.14,0.71,0.01,NaN


### Numer of Reps done to termination criterion

In [11]:
# Sets done = Number of sets done last set included
# total_reps = total sum of reps


absolvierte_reps = pd.read_csv("https://media.githubusercontent.com/media/bvn3141/beta_ala_study/main/Absolvierte_reps.csv?token=BGUXDREQFMT374SDX7E4M7DF6AF56",
                                sep=';')
absolvierte_reps = absolvierte_reps.iloc[:24, :17]
absolvierte_reps.drop('Unnamed: 8', axis=1, inplace=True)
absolvierte_reps

,id,group,sets_done_pre,reps_done_last set_pre,total_reps_pre,sets_done_post,reps_done_last_set_post,total_reps_post,letzte_peaks_1_pre,letzte_peaks_2_pre,letzte_peaks_3_pre,Mean_last_peaks_pre,letzte_peaks_1_post,letzte_peaks_2_post,letzte_peaks_3_post,Mean_last_peaks_post
0,15073.00,beta_ala,5.00,15.00,75.00,6.00,13.00,88.00,127.50,122.30,119.30,123.03,125.70,123.80,115.50,121.67
1,20438.00,beta_ala,2.00,12.00,27.00,7.00,13.00,103.00,146.70,146.30,143.70,145.57,140.00,122.30,123.00,128.43
2,25778.00,beta_ala,4.00,15.00,60.00,2.00,14.00,29.00,75.80,77.00,80.70,77.83,70.50,68.70,62.30,67.17
3,26107.00,beta_ala,8.00,15.00,120.00,6.00,14.00,89.00,101.70,93.80,87.00,94.17,95.00,94.50,87.00,92.17
4,27351.00,beta_ala,2.00,14.00,29.00,1.00,14.00,14.00,56.30,75.00,72.80,68.03,74.30,71.30,65.00,70.20
5,28514.00,placebo,7.00,15.00,105.00,7.00,9.00,99.00,57.80,63.80,65.70,62.43,57.50,53.70,62.30,57.83
6,33468.00,placebo,3.00,15.00,45.00,2.00,14.00,29.00,70.50,74.30,72.80,72.53,81.80,75.80,80.30,79.30
7,39337.00,placebo,3.00,14.00,44.00,4.00,15.00,60.00,149.30,146.30,141.50,145.70,109.50,117.50,117.50,114.83
8,39750.00,placebo,2.00,15.00,30.00,4.00,12.00,57.00,71.30,65.30,60.50,65.70,71.70,56.70,60.50,62.97
9,44858.00,beta_ala,3.00,14.00,44.00,4.00,13.00,58.00,72.00,63.80,61.50,65.77,80.00,68.70,73.50,74.07


In [12]:

absolvierte_reps_l = pd.melt(absolvierte_reps[['total_reps_pre', 'total_reps_post', 'id', 'group']], 
                             id_vars=['id', 'group'], var_name = 'measurement', value_name='reps done')

In [13]:
(
    ggplot(absolvierte_reps_l, aes(x='measurement', y='reps done'))
    + geom_boxplot(aes(color='group', group='group'))
    + labs(title='Completed reps until termation')
    + ylab('Reps total')
)

In [14]:
# statistic of total reps done

rep_stat = pd.read_csv("https://media.githubusercontent.com/media/bvn3141/beta_ala_study/main/Absolvierte_reps_sheet3.csv?token=BGUXDRB2SZ353MF34F76MRTF6AHYM", 
                         sep = ';')

reps_stat = pg.mixed_anova(dv='reps',between='group', within='experiment', subject='id', data=rep_stat, correction='auto', effsize='np2')
reps_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,784.08,1,22,784.08,0.74,0.40,0.03,NaN
1,experiment,2.08,1,22,2.08,0.01,0.94,0.00,1.00
2,Interaction,108.00,1,22,108.00,0.32,0.58,0.01,NaN


## Calculate work done in pre and post test as integral over the complete range of motion.


In [15]:
# extract ids
probanden_ids = iso['id'].unique()

# create data-frame
total_work_done = pd.DataFrame(columns=['id', 'experiment', 'group', 'work_done'])

for proband_id in probanden_ids:
    proband_data = iso[iso['id'] == proband_id]
    
    # group information
    group = proband_data['group'].iloc[0]
    
    # filter for pre and post-test
    proband_data_pre = proband_data[proband_data['experiment'] == 'pretest'].copy()
    proband_data_post = proband_data[proband_data['experiment'] == 'posttest'].copy()

    # adjust time column
    proband_data_pre.loc[:, 'time_ms'] = proband_data_pre.index * 5 + 5
    proband_data_post.reset_index(drop=True, inplace=True)
    proband_data_post.loc[:, 'time_ms'] = proband_data_post.index * 5 + 5

    # calculate position change
    proband_data_pre['displacement_deg_or_mm'] = proband_data_pre['pos_deg_or_mm'].diff()
    proband_data_post['displacement_deg_or_mm'] = proband_data_post['pos_deg_or_mm'].diff()
    proband_data_pre['displacement_deg_or_mm'] = proband_data_pre['displacement_deg_or_mm'].where(proband_data_pre['displacement_deg_or_mm'].abs() <= 0.6, other=np.nan)
    proband_data_post['displacement_deg_or_mm'] = proband_data_post['displacement_deg_or_mm'].where(proband_data_post['displacement_deg_or_mm'].abs() <= 0.6, other=np.nan)
    
    # sum position change as total distance
    gesamtstrecke_mm_pre = proband_data_pre['displacement_deg_or_mm'].abs().sum()
    gesamtstrecke_m_pre = gesamtstrecke_mm_pre / 1000  # Auf Meter umgerechnet
    gesamtstrecke_mm_post = proband_data_post['displacement_deg_or_mm'].abs().sum()
    gesamtstrecke_m_post = gesamtstrecke_mm_post / 1000  # Auf Meter umgerechnet
    
    # calculation of time
    proband_data_pre['gesamtstrecke_m'] = proband_data_pre.index * (gesamtstrecke_m_pre / (len(proband_data_pre) - 1))
    proband_data_post['gesamtstrecke_m'] = proband_data_post.index * (gesamtstrecke_m_post / (len(proband_data_post) - 1))

    # calculation of torque values
    proband_data_pre['torque_nm_abs'] = proband_data_pre['torque_nm'].abs()
    proband_data_post['torque_nm_abs'] = proband_data_post['torque_nm'].abs()

    # calculation of work
    work_done_pre = trapz(proband_data_pre['torque_nm_abs'], proband_data_pre['gesamtstrecke_m'])
    work_done_post = trapz(proband_data_post['torque_nm_abs'], proband_data_post['gesamtstrecke_m'])

    # add results
    total_work_done = pd.concat([
        total_work_done,
        pd.DataFrame({
            'id': [proband_id, proband_id],
            'experiment': ['pretest', 'posttest'],
            'group': [group, group],
            'work_done': [work_done_pre, work_done_post]
        })
    ], ignore_index=True)



C:\Users\bvenn\AppData\Local\Temp\ipykernel_11308\20225016.py:47: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  total_work_done = pd.concat([


In [16]:
total_work_done.head()

,id,experiment,group,work_done
0,15073,pretest,beta_ala,1386.94
1,15073,posttest,beta_ala,1598.20
2,20438,pretest,beta_ala,1042.87
3,20438,posttest,beta_ala,1931.84
4,25778,pretest,beta_ala,1196.42


# Calculate work from Warmup to termination of protocol

In [17]:
# reset index
iso_reset = iso.reset_index(drop=True)

# filter conditions
pretest_conditions = (
    # pretest
    (iso_reset['id'] == 20438) & (iso_reset['measurement'] == 's2') & (iso_reset['experiment'] == 'pretest') & (iso_reset['rep'] > 24) |
    (iso_reset['id'] == 27351) & (iso_reset['measurement'] == 's2') & (iso_reset['experiment'] == 'pretest') & (iso_reset['rep'] > 28) |
    (iso_reset['id'] == 39337) & (iso_reset['measurement'] == 's3') & (iso_reset['experiment'] == 'pretest') & (iso_reset['rep'] > 28) |
    (iso_reset['id'] == 44858) & (iso_reset['measurement'] == 's3') & (iso_reset['experiment'] == 'pretest') & (iso_reset['rep'] > 28) |
    (iso_reset['id'] == 49444) & (iso_reset['measurement'] == 's3') & (iso_reset['experiment'] == 'pretest') & (iso_reset['rep'] > 28) |
    (iso_reset['id'] == 50843) & (iso_reset['measurement'] == 's3') & (iso_reset['experiment'] == 'pretest') & (iso_reset['rep'] > 24) |
    (iso_reset['id'] == 54145) & (iso_reset['measurement'] == 's3') & (iso_reset['experiment'] == 'pretest') & (iso_reset['rep'] > 26) |
    (iso_reset['id'] == 54658) & (iso_reset['measurement'] == 's4') & (iso_reset['experiment'] == 'pretest') & (iso_reset['rep'] > 22) |
    (iso_reset['id'] == 58046) & (iso_reset['measurement'] == 's3') & (iso_reset['experiment'] == 'pretest') & (iso_reset['rep'] > 28) |
    (iso_reset['id'] == 59289) & (iso_reset['measurement'] == 's4') & (iso_reset['experiment'] == 'pretest') & (iso_reset['rep'] > 20) |
    (iso_reset['id'] == 61104) & (iso_reset['measurement'] == 's2') & (iso_reset['experiment'] == 'pretest') & (iso_reset['rep'] > 26) |
    (iso_reset['id'] == 71620) & (iso_reset['measurement'] == 's2') & (iso_reset['experiment'] == 'pretest') & (iso_reset['rep'] > 22) |
    (iso_reset['id'] == 73162) & (iso_reset['measurement'] == 's3') & (iso_reset['experiment'] == 'pretest') & (iso_reset['rep'] > 20) |
    (iso_reset['id'] == 76847) & (iso_reset['measurement'] == 's2') & (iso_reset['experiment'] == 'pretest') & (iso_reset['rep'] > 22)
)

posttest_conditions = (
    # posttest
    (iso_reset['id'] == 15073) & (iso_reset['measurement'] == 's6') & (iso_reset['experiment'] == 'posttest') & (iso_reset['rep'] > 26) |
    (iso_reset['id'] == 20438) & (iso_reset['measurement'] == 's7') & (iso_reset['experiment'] == 'posttest') & (iso_reset['rep'] > 26) |
    (iso_reset['id'] == 25778) & (iso_reset['measurement'] == 's2') & (iso_reset['experiment'] == 'posttest') & (iso_reset['rep'] > 28) |
    (iso_reset['id'] == 26107) & (iso_reset['measurement'] == 's6') & (iso_reset['experiment'] == 'posttest') & (iso_reset['rep'] > 28) |
    (iso_reset['id'] == 27351) & (iso_reset['measurement'] == 's1') & (iso_reset['experiment'] == 'posttest') & (iso_reset['rep'] > 28) |
    (iso_reset['id'] == 28514) & (iso_reset['measurement'] == 's7') & (iso_reset['experiment'] == 'posttest') & (iso_reset['rep'] > 18) |
    (iso_reset['id'] == 33468) & (iso_reset['measurement'] == 's2') & (iso_reset['experiment'] == 'posttest') & (iso_reset['rep'] > 28) |
    (iso_reset['id'] == 39750) & (iso_reset['measurement'] == 's4') & (iso_reset['experiment'] == 'posttest') & (iso_reset['rep'] > 24) |
    (iso_reset['id'] == 44858) & (iso_reset['measurement'] == 's4') & (iso_reset['experiment'] == 'posttest') & (iso_reset['rep'] > 26) |
    (iso_reset['id'] == 49830) & (iso_reset['measurement'] == 's4') & (iso_reset['experiment'] == 'posttest') & (iso_reset['rep'] > 22) |
    (iso_reset['id'] == 50843) & (iso_reset['measurement'] == 's3') & (iso_reset['experiment'] == 'posttest') & (iso_reset['rep'] > 26) |
    (iso_reset['id'] == 54145) & (iso_reset['measurement'] == 's3') & (iso_reset['experiment'] == 'posttest') & (iso_reset['rep'] > 28) |
    (iso_reset['id'] == 59289) & (iso_reset['measurement'] == 's3') & (iso_reset['experiment'] == 'posttest') & (iso_reset['rep'] > 16) |
    (iso_reset['id'] == 61104) & (iso_reset['measurement'] == 's2') & (iso_reset['experiment'] == 'posttest') & (iso_reset['rep'] > 22) |
    (iso_reset['id'] == 76847) & (iso_reset['measurement'] == 's2') & (iso_reset['experiment'] == 'posttest') & (iso_reset['rep'] > 26) |
    (iso_reset['id'] == 93302) & (iso_reset['measurement'] == 's1') & (iso_reset['experiment'] == 'posttest') & (iso_reset['rep'] > 26) |
    (iso_reset['id'] == 93672) & (iso_reset['measurement'] == 's2') & (iso_reset['experiment'] == 'posttest') & (iso_reset['rep'] > 28)
)

# combined conditions
combined_conditions = pretest_conditions | posttest_conditions

# use conditions and filter frame
warmup_abbruch = iso_reset[~combined_conditions].reset_index(drop=True)


In [18]:
def calculate_work_done(df):
    # extraxt ids
    probanden_ids = df['id'].unique()

    # create dataframe for work
    total_work_done = pd.DataFrame(columns=['id', 'experiment', 'group', 'work_done'])

    for proband_id in probanden_ids:
        proband_data = df[df['id'] == proband_id]

        # group information
        group = proband_data['group'].iloc[0]
    
        # filter for pre and posttest
        proband_data_pre = proband_data[proband_data['experiment'] == 'pretest'].copy()
        proband_data_post = proband_data[proband_data['experiment'] == 'posttest'].copy()

        # adjust time
        proband_data_pre.loc[:, 'time_ms'] = proband_data_pre.index * 5 + 5
        proband_data_post.reset_index(drop=True, inplace=True)
        proband_data_post.loc[:, 'time_ms'] = proband_data_post.index * 5 + 5

        # position change pre
        proband_data_pre['displacement_deg_or_mm'] = proband_data_pre['pos_deg_or_mm'].diff()
        proband_data_post['displacement_deg_or_mm'] = proband_data_post['pos_deg_or_mm'].diff()
        proband_data_pre['displacement_deg_or_mm'] = proband_data_pre['displacement_deg_or_mm'].where(proband_data_pre['displacement_deg_or_mm'].abs() <= 0.6, other=np.nan)
        proband_data_post['displacement_deg_or_mm'] = proband_data_post['displacement_deg_or_mm'].where(proband_data_post['displacement_deg_or_mm'].abs() <= 0.6, other=np.nan)
        
        # total distance pre
        gesamtstrecke_mm_pre = proband_data_pre['displacement_deg_or_mm'].abs().sum()
        gesamtstrecke_m_pre = gesamtstrecke_mm_pre / 1000  # Auf Meter umgerechnet
        gesamtstrecke_mm_post = proband_data_post['displacement_deg_or_mm'].abs().sum()
        gesamtstrecke_m_post = gesamtstrecke_mm_post / 1000  # Auf Meter umgerechnet
        
        # time
        proband_data_pre['gesamtstrecke_m'] = proband_data_pre.index * (gesamtstrecke_m_pre / (len(proband_data_pre) - 1))
        proband_data_post['gesamtstrecke_m'] = proband_data_post.index * (gesamtstrecke_m_post / (len(proband_data_post) - 1))

        # torque values
        proband_data_pre['torque_nm_abs'] = proband_data_pre['torque_nm'].abs()
        proband_data_post['torque_nm_abs'] = proband_data_post['torque_nm'].abs()

        # calculate work
        work_done_pre = trapz(proband_data_pre['torque_nm_abs'], proband_data_pre['gesamtstrecke_m'])
        work_done_post = trapz(proband_data_post['torque_nm_abs'], proband_data_post['gesamtstrecke_m'])

        # add results to frame
        total_work_done = pd.concat([
            total_work_done,
            pd.DataFrame({
                'id': [proband_id, proband_id],
                'experiment': ['pretest', 'posttest'],
                'group': [group, group],
                'work_done': [work_done_pre, work_done_post]
            })
        ], ignore_index=True)

    return total_work_done

# use for warmup_abbruch
total_work_done_warmup_abbruch = calculate_work_done(warmup_abbruch)


C:\Users\bvenn\AppData\Local\Temp\ipykernel_11308\3163125762.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  total_work_done = pd.concat([


In [19]:
total_work_done_warmup_abbruch.head()

,id,experiment,group,work_done
0,15073,pretest,beta_ala,1386.94
1,15073,posttest,beta_ala,1582.15
2,20438,pretest,beta_ala,1013.77
3,20438,posttest,beta_ala,1914.85
4,25778,pretest,beta_ala,1196.42


In [20]:
work_pre = total_work_done_warmup_abbruch[total_work_done_warmup_abbruch['experiment']=='pretest'].reset_index(drop=True)
work_post = total_work_done_warmup_abbruch[total_work_done_warmup_abbruch['experiment']=='posttest'].reset_index(drop=True)

In [21]:
work_pre['work_diff'] = work_post['work_done'] - work_pre['work_done']
work_pre

,id,experiment,group,work_done,work_diff
0,15073,pretest,beta_ala,1386.94,195.22
1,20438,pretest,beta_ala,1013.77,901.08
2,25778,pretest,beta_ala,1196.42,-457.67
3,26107,pretest,beta_ala,1902.48,-339.16
4,27351,pretest,beta_ala,487.11,-113.34
5,28514,pretest,placebo,1018.59,14.31
6,93672,pretest,beta_ala,909.05,-6.84
7,33468,pretest,placebo,694.33,-137.90
8,39337,pretest,placebo,1102.42,204.87
9,39750,pretest,placebo,584.58,207.18


In [22]:
(
    ggplot(work_pre, aes(x='group', y='work_diff', color='group'))
    + geom_boxplot()
    + labs(title='Pre to Post Work difference', subtitle='Set one to termination')
    + ylab('Work Difference [J]')
)

In [23]:
x = work_pre.query('group == "placebo"')['work_diff']
y = work_pre.query('group == "beta_ala"')['work_diff']

In [24]:
work_stat = pg.ttest(x, y, paired=False, alternative='less', correction='auto')
work_stat

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-0.32,22,less,0.38,"[-inf, 178.86]",0.13,0.775,0.09


In [188]:
total_work_done_warmup_abbruch.to_

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          48 non-null     object 
 1   experiment  48 non-null     object 
 2   group       48 non-null     object 
 3   work_done   48 non-null     float64
dtypes: float64(1), object(3)
memory usage: 1.6+ KB


In [25]:
(
    ggplot(total_work_done_warmup_abbruch, aes(x='experiment', y='work_done'))
    + geom_boxplot(aes(group='group', color='group'))
    + labs(title='Total Work Done: Warm-up to termination')
    + ylab('Work [J]')
    + xlab('measurement')
)

In [26]:
work_wu_abb_stat = pg.mixed_anova(dv='work_done',between='group', within='experiment', subject='id', data=total_work_done_warmup_abbruch, correction='auto', effsize='np2')
work_wu_abb_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,844446.70,1,22,844446.70,4.10,0.06,0.16,NaN
1,experiment,289.60,1,22,289.60,0.01,0.94,0.00,1.00
2,Interaction,5058.09,1,22,5058.09,0.10,0.75,0.00,NaN


In [27]:
posthoc_work_wu_abb = pg.pairwise_tests(data=total_work_done_warmup_abbruch, dv='work_done', between='group', subject='id', parametric=True, padjust='bonf', effsize='cohen')
posthoc_work_wu_abb

,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,cohen
0,group,beta_ala,placebo,False,True,2.63,46.00,two-sided,0.01,4.342,0.76


## Supplemental Plots and calculations for work done by time point

In [28]:
def calculate_work_done_per_measurement(df):
    # Eindeutige Probanden-IDs extrahieren
    probanden_ids = df['id'].unique()

    # DataFrame für die gesamte Arbeit erstellen
    total_work_done = pd.DataFrame(columns=['id', 'experiment', 'group', 'measurement', 'work_done'])

    for proband_id in probanden_ids:
        proband_data = df[df['id'] == proband_id]

        # Gruppeninfo abgreifen
        group = proband_data['group'].iloc[0]

        # Eindeutige Messzeitpunkte extrahieren
        measurements = proband_data['measurement'].unique()

        for measurement in measurements:
            measurement_data = proband_data[proband_data['measurement'] == measurement].copy()

            # Zeit-Spalte anpassen für den Messzeitpunkt
            measurement_data.loc[:, 'time_ms'] = measurement_data.index * 5 + 5

            # Index zurücksetzen, um wieder bei 5ms zu beginnen
            measurement_data.reset_index(drop=True, inplace=True)
            measurement_data.loc[:, 'time_ms'] = measurement_data.index * 5 + 5

            # Berechnung der Positionsänderung
            measurement_data['displacement_deg_or_mm'] = measurement_data['pos_deg_or_mm'].diff()

            # Filtere Werte über 0.7 aus der Spalte
            measurement_data['displacement_deg_or_mm'] = measurement_data['displacement_deg_or_mm'].where(
                measurement_data['displacement_deg_or_mm'].abs() <= 0.6, other=np.nan
            )

            # Summierte Positionsänderungen als Gesamtstrecke
            gesamtstrecke_mm = measurement_data['displacement_deg_or_mm'].abs().sum()
            gesamtstrecke_m = gesamtstrecke_mm / 1000  # Auf Meter umgerechnet

            # Berechnung der normierten Zeit
            measurement_data['gesamtstrecke_m'] = measurement_data.index * (gesamtstrecke_m / (len(measurement_data) - 1))

            # Berechnung absoluter Torque Werte, um Drehmoment in jede Richtung zu erfassen
            measurement_data['torque_nm_abs'] = measurement_data['torque_nm'].abs()

            # Berechnung der Arbeit
            work_done = trapz(measurement_data['torque_nm_abs'], measurement_data['gesamtstrecke_m'])

            # Ergebnisse zum total_work_done DataFrame hinzufügen
            total_work_done = pd.concat([
                total_work_done,
                pd.DataFrame({
                    'id': [proband_id],
                    'experiment': [measurement_data['experiment'].iloc[0]],
                    'group': [group],
                    'measurement': [measurement],
                    'work_done': [work_done]
                })
            ], ignore_index=True)

    return total_work_done

# Anwenden auf warmup_abbruch
total_work_done_per_measurement = calculate_work_done_per_measurement(warmup_abbruch)


C:\Users\bvenn\AppData\Local\Temp\ipykernel_11308\680265839.py:49: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  total_work_done = pd.concat([


In [29]:
total_work_done_posttest = calculate_work_done_per_measurement(warmup_abbruch[warmup_abbruch['experiment'] == 'posttest'])

C:\Users\bvenn\AppData\Local\Temp\ipykernel_11308\680265839.py:49: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  total_work_done = pd.concat([


In [30]:
total_work_done_pretest = total_work_done_per_measurement.query('experiment == "pretest"')

In [31]:

grouped_means_pre = total_work_done_pretest.groupby(['group', 'measurement'], observed=False).agg({'work_done':'mean'}).reset_index()
grouped_std_pre = total_work_done_pretest.groupby(['group', 'measurement'], observed=False).agg({'work_done':'std'}).reset_index()

grouped_means_post = total_work_done_posttest.groupby(['group', 'measurement'], observed=False).agg({'work_done':'mean'}).reset_index()
grouped_std_post = total_work_done_posttest.groupby(['group', 'measurement'], observed=False).agg({'work_done':'std'}).reset_index()

# rename columns
grouped_means_pre.rename(columns={
    'work_done': 'work_done_mean',
}, inplace=True)

grouped_means_post.rename(columns={
    'work_done': 'work_done_mean',
}, inplace=True)

grouped_std_pre.rename(columns={
    'work_done': 'work_done_std',
}, inplace=True)

grouped_std_post.rename(columns={
    'work_done': 'work_done_std',
}, inplace=True)

In [32]:
grouped_means_pre = pd.concat([grouped_means_pre, grouped_std_pre.drop(['group', 'measurement'], axis=1)], axis=1)
grouped_means_post = pd.concat([grouped_means_post, grouped_std_post.drop(['group', 'measurement'], axis=1)], axis=1)

In [33]:
count_data = total_work_done_pretest.groupby(['group', 'measurement'])['work_done'].count().reset_index(name='count')

grouped_means_pre_with_count = pd.merge(grouped_means_pre, count_data, on=['group', 'measurement'])

# Plot erstellen
(
    ggplot(grouped_means_pre_with_count, aes(x='measurement', y='work_done_mean', color='group', group='group'))
    + geom_point(position=position_dodge(width=0.4))
    + geom_line(position=position_dodge(width=0.4))
    + geom_errorbar(
        aes(
            ymin=grouped_means_pre_with_count['work_done_mean'] - grouped_means_pre_with_count['work_done_std'],
            ymax=grouped_means_pre_with_count['work_done_mean'] + grouped_means_pre_with_count['work_done_std'],
            fill='group'
        ),
        position=position_dodge(width=0.4),
        linetype="blank"
    )
    + geom_text(
        aes(label='count'),
        position=position_dodge(width=0.4),
        nudge_x=0.4,
        vjust=-0.5,
        size=8
    )
    + labs(
        title='Pre-Test: means and std of work by group',
        subtitle='Number represents the amount of participants in the respective set'
    )
    + ylab('Work Done [J]')
    + xlab('Measurement')
    + scale_x_discrete(limits=['warmup', 'pt', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8'])
)


In [34]:
count_data = total_work_done_posttest.groupby(['group', 'measurement'])['work_done'].count().reset_index(name='count')
grouped_means_post_with_count = pd.merge(grouped_means_post, count_data, on=['group', 'measurement'])


(
    ggplot(grouped_means_post_with_count, aes(x='measurement', y='work_done_mean', color='group', group='group'))
    + geom_point(position=position_dodge(width=0.4))
    + geom_line(position=position_dodge(width=0.4))
    + geom_errorbar(
        aes(
            ymin=grouped_means_post_with_count['work_done_mean'] - grouped_means_post_with_count['work_done_std'],
            ymax=grouped_means_post_with_count['work_done_mean'] + grouped_means_post_with_count['work_done_std'],
            fill='group'
        ),
        position=position_dodge(width=0.4),
        linetype="blank"
    )
    + geom_text(
        aes(label='count'),
        position=position_dodge(width=0.4),
        nudge_x=0.4,
        vjust=-0.5,
        size=8
    )
    + labs(
        title='Post-Test: Means and std of work done bx group',
        subtitle='number represents the amount of participants in the respective set'
    )
    + ylab('Work Done [J]')
    + xlab('Measurement')
    + scale_x_discrete(limits=['warmup', 'pt', 's1', 's2', 's3', 's4', 's5', 's6', 's7'])
)


In [35]:
my_colors = ['#FF5733', '#33FF57', '#3366FF', '#FFFF33', '#33FFFF', '#FF33FF', '#33CC00',
             '#FF33CC', '#FF6633', '#33FF00', '#6600CC', '#33CC33', '#FF00FF', '#6600FF',
             '#00CCFF', '#CC33CC', '#0066FF', '#33CC99', '#66CC00', '#996633', '#00FF33',
             '#CC0033', '#33CC66', '#CC66FF']

(
    ggplot(total_work_done_per_measurement.query('experiment == "pretest"'), aes(x='measurement', y='work_done', color='id', group='id'))
    + geom_point()
    + geom_line()
    + scale_color_manual(values=my_colors)
    + labs(title='Individual work done in Pre-Test', subtitle = 'color = id')
    + theme(legend_position='none')
    + ylab('Work Done [J]')
    + xlab('measurement')
)

In [36]:
my_colors = ['#FF5733', '#33FF57', '#3366FF', '#FFFF33', '#33FFFF', '#FF33FF', '#33CC00',
             '#FF33CC', '#FF6633', '#33FF00', '#6600CC', '#33CC33', '#FF00FF', '#6600FF',
             '#00CCFF', '#CC33CC', '#0066FF', '#33CC99', '#66CC00', '#996633', '#00FF33',
             '#CC0033', '#33CC66', '#CC66FF']

(
    ggplot(total_work_done_posttest, aes(x='measurement', y='work_done', color='id', group='id'))
    + geom_point()
    + geom_line()
    + scale_color_manual(values=my_colors)
    + labs(title='Individual work done in Post-Test', subtitle = 'color = id')
    + theme(legend_position='none')
    + ylab('Work Done [J]')
    + xlab('measurement')
)

In [37]:
# number of participants in each set
participants_count_per_measurement = total_work_done_per_measurement.query('experiment == "pretest"').groupby('measurement')['id'].nunique().reset_index()

participants_count_per_measurement.columns = ['measurement', 'num_participants']

print(participants_count_per_measurement)

  measurement  num_participants
0          pt                24
1          s1                24
2          s2                24
3          s3                16
4          s4                 8
5          s5                 4
6          s6                 3
7          s7                 2
8          s8                 1
9      warmup                24


## Supplemental Material. Work done from peak torque to termination.

In [38]:
total_work_done_filtered = warmup_abbruch[warmup_abbruch['measurement'] != 'warmup'].reset_index(drop=True)

total_work_done_pt_abbruch = calculate_work_done(total_work_done_filtered)

C:\Users\bvenn\AppData\Local\Temp\ipykernel_11308\3163125762.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  total_work_done = pd.concat([


In [39]:
(
    ggplot(total_work_done_pt_abbruch, aes(x='experiment', y='work_done'))
    + geom_boxplot(aes(group='group', color='group'))
    + labs(title='Total Work Done: Peak Torque Test to termination')
    + ylab('Work in Joule')
    + xlab('Measurement')
)

In [40]:
work_pt_abb_stat = pg.mixed_anova(dv='work_done',between='group', within='experiment', subject='id', data=total_work_done_pt_abbruch, correction='auto', effsize='np2')
work_pt_abb_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,766708.19,1,22,766708.19,4.99,0.04,0.18,NaN
1,experiment,12.96,1,22,12.96,0.00,0.99,0.00,1.00
2,Interaction,7790.76,1,22,7790.76,0.15,0.70,0.01,NaN


In [41]:
posthoc_pt_abb_stat = pg.pairwise_tests(data=total_work_done_pt_abbruch, dv='work_done', between='group', subject='id', parametric=True, padjust='bonf', effsize='cohen')
posthoc_pt_abb_stat

,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,cohen
0,group,beta_ala,placebo,False,True,2.80,46.00,two-sided,0.01,6.113,0.81


### Work done from fatigue set one to termination of exercise.

In [42]:
# filter warmup and peak torque
s1_abbruch = warmup_abbruch[(warmup_abbruch['measurement'] != 'warmup') & (warmup_abbruch['measurement'] != 'pt')].reset_index(drop=True)

# use function
total_work_done_s1_abbruch = calculate_work_done(s1_abbruch)

C:\Users\bvenn\AppData\Local\Temp\ipykernel_11308\3163125762.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  total_work_done = pd.concat([


In [43]:
(
    ggplot(total_work_done_s1_abbruch, aes(x='experiment', y='work_done'))
    + geom_boxplot(aes(group='group', color='group'))
    + labs(title='Total Work Done: set one to termination')
    + ylab('Work in Joule')
    + xlab('Measurement')
)

In [44]:
work_s1_abb_stat = pg.mixed_anova(dv='work_done',between='group', within='experiment', subject='id', data=total_work_done_s1_abbruch, correction='auto', effsize='np2')
work_s1_abb_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,628394.71,1,22,628394.71,4.30,0.05,0.16,NaN
1,experiment,252.88,1,22,252.88,0.01,0.94,0.00,1.00
2,Interaction,13794.99,1,22,13794.99,0.29,0.60,0.01,NaN


In [45]:
posthoc_work_s1_abb = pg.pairwise_tests(data=total_work_done_s1_abbruch, dv='work_done', between='group', subject='id', parametric=True, padjust='bonf', effsize='cohen')
posthoc_work_s1_abb

,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,cohen
0,group,beta_ala,placebo,False,True,2.60,46.00,two-sided,0.01,4.052,0.75


# Determine fatigue index from set one to termination

In [46]:
s1_abbruch

,id,measurement,time_ms,pos_deg_or_mm,torque_nm,speed_deg/s_or_mm/s,torque_without_comp,rep,set,torque_dyn,force_right,force_left,experiment,group
0,15073,s1,5,11.40,21.80,0.00,72,1,1,-218,0,0,pretest,beta_ala
1,15073,s1,10,11.60,21.80,0.30,68,1,1,-218,0,0,pretest,beta_ala
2,15073,s1,15,11.60,22.50,1.80,65,1,1,-225,0,0,pretest,beta_ala
3,15073,s1,20,11.40,24.00,4.30,57,1,1,-240,0,0,pretest,beta_ala
4,15073,s1,25,11.40,26.70,5.80,53,1,1,-267,0,0,pretest,beta_ala
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1473081,93672,s2,43210,11.30,15.50,-15.30,150,28,1,-155,0,0,posttest,beta_ala
1473082,93672,s2,43215,11.10,20.30,-10.00,102,28,1,-203,0,0,posttest,beta_ala
1473083,93672,s2,43220,11.10,25.20,-8.30,53,28,1,-252,0,0,posttest,beta_ala
1473084,93672,s2,43225,11.10,30.00,-8.00,5,28,1,-300,0,0,posttest,beta_ala


## mean of top thee torque peaks

In [47]:
peaks_largest = []

for _, sub_df in iso.groupby(['id', 'experiment', 'group']):
    proband_id, experiment_type, group = _
    
    px = sub_df.query('rep % 2 == 0').copy()
    
    px['torque_nm_abs'] = px['torque_nm'].abs()
    px.loc[:, 'time_ms'] = px.index * 5 + 5
    peaks, properties = find_peaks(px['torque_nm_abs'], height=50, distance=200)
    top_three_peaks = px['torque_nm_abs'].iloc[peaks].nlargest(3)
    mean_top_three = top_three_peaks.mean()
    
    
    # save results
    peaks_largest.append({
        'id': proband_id,
        'experiment': experiment_type,
        'group': group,
        'top_three_peaks': top_three_peaks.tolist(),
        'mean_top_three': mean_top_three,
    })

# to dataframe
peaks_largest = pd.DataFrame(peaks_largest)



C:\Users\bvenn\AppData\Local\Temp\ipykernel_11308\3135058859.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for _, sub_df in iso.groupby(['id', 'experiment', 'group']):


## mean of peaks from last three reps

In [48]:
# to open the file download the Absolvierte_reps.xlsx file and copy its path within the brackets
means_last_reps = pd.read_csv("https://media.githubusercontent.com/media/bvn3141/beta_ala_study/main/Absolvierte_reps_sheet2.csv?token=BGUXDREAUUMCSPNZ6RVLV5TF6AID6",
                                sep=';')
means_last_reps

,id,group,experiment,letzte_peaks_1,letzte_peaks_2,letzte_peaks_3,Mean_peaks
0,15073,beta_ala,pretest,127.50,122.30,119.30,123.03
1,20438,beta_ala,pretest,146.70,146.30,143.70,145.57
2,25778,beta_ala,pretest,75.80,77.00,80.70,77.83
3,26107,beta_ala,pretest,101.70,93.80,87.00,94.17
4,27351,beta_ala,pretest,56.30,75.00,72.80,68.03
5,28514,placebo,pretest,57.80,63.80,65.70,62.43
6,33468,placebo,pretest,70.50,74.30,72.80,72.53
7,39337,placebo,pretest,149.30,146.30,141.50,145.70
8,39750,placebo,pretest,71.30,65.30,60.50,65.70
9,44858,beta_ala,pretest,72.00,63.80,61.50,65.77


In [49]:
peaks_largest = pd.merge(peaks_largest, means_last_reps[['experiment', 'id', 'group', 'Mean_peaks']], 
                         on=['experiment', 'id', 'group'], how='left')



### calculate fatigue index

In [50]:
peaks_largest['fatigue_index'] = ((peaks_largest['mean_top_three'] - peaks_largest['Mean_peaks']) / 
                                peaks_largest['mean_top_three']) * 100
peaks_largest

,id,experiment,group,top_three_peaks,mean_top_three,Mean_peaks,fatigue_index
0,15073,posttest,beta_ala,"[269.0, 264.8, 249.0]",260.93,121.67,53.37
1,15073,pretest,beta_ala,"[249.5, 248.0, 243.0]",246.83,123.03,50.16
2,20438,posttest,beta_ala,"[291.0, 284.7, 284.0]",286.57,128.43,55.18
3,20438,pretest,beta_ala,"[306.8, 294.0, 292.2]",297.67,145.57,51.10
4,25778,posttest,beta_ala,"[155.7, 152.3, 147.5]",151.83,67.17,55.76
5,25778,pretest,beta_ala,"[163.2, 154.5, 145.5]",154.40,77.83,49.59
6,26107,posttest,beta_ala,"[195.0, 194.0, 186.5]",191.83,92.17,51.95
7,26107,pretest,beta_ala,"[201.5, 186.8, 182.0]",190.10,94.17,50.46
8,27351,posttest,beta_ala,"[166.5, 161.0, 156.0]",161.17,70.20,56.44
9,27351,pretest,beta_ala,"[162.8, 162.8, 155.0]",160.20,68.03,57.53


## pre to post difference of fatigue index

In [51]:
fatigue_pre = peaks_largest[peaks_largest['experiment'] == 'pretest'].reset_index(drop=True)
fatigue_post = peaks_largest[peaks_largest['experiment'] == 'posttest'].reset_index(drop=True)

In [52]:
fatigue_pre['diff_fatigue_index'] = fatigue_post['fatigue_index'] - fatigue_pre['fatigue_index']
fatigue_pre

,id,experiment,group,top_three_peaks,mean_top_three,Mean_peaks,fatigue_index,diff_fatigue_index
0,15073,pretest,beta_ala,"[249.5, 248.0, 243.0]",246.83,123.03,50.16,3.21
1,20438,pretest,beta_ala,"[306.8, 294.0, 292.2]",297.67,145.57,51.10,4.09
2,25778,pretest,beta_ala,"[163.2, 154.5, 145.5]",154.40,77.83,49.59,6.17
3,26107,pretest,beta_ala,"[201.5, 186.8, 182.0]",190.10,94.17,50.46,1.49
4,27351,pretest,beta_ala,"[162.8, 162.8, 155.0]",160.20,68.03,57.53,-1.09
5,28514,pretest,placebo,"[121.2, 119.0, 118.2]",119.47,62.43,47.74,8.18
6,33468,pretest,placebo,"[158.0, 157.5, 156.5]",157.33,72.53,53.90,-2.68
7,39337,pretest,placebo,"[296.7, 274.2, 268.5]",279.80,145.70,47.93,7.41
8,39750,pretest,placebo,"[139.2, 135.5, 131.7]",135.47,65.70,51.50,-0.06
9,44858,pretest,beta_ala,"[143.3, 142.2, 140.7]",142.07,65.77,53.70,-3.61


In [53]:
(
    ggplot(fatigue_pre, aes(x='group', y='diff_fatigue_index', color='group'))
    + geom_boxplot()
    + ylab('Difference of Fatigue Index')
)

In [54]:
x = fatigue_pre.query('group == "placebo"')['diff_fatigue_index']
y = fatigue_pre.query('group == "beta_ala"')['diff_fatigue_index']

work_stat = pg.ttest(x, y, paired=False, alternative='less', correction='auto')
work_stat

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.07,22,less,0.85,"[-inf, 5.67]",0.44,0.884,0.00


# TMG Analysis

In [56]:
tmg_raw = pd.read_csv("https://media.githubusercontent.com/media/bvn3141/beta_ala_study/main/tmg_raw_data.csv?token=BGUXDRHQDAF63FB23H7OIK3F6AJ7A")
tmg_raw.rename(columns={'Unnamed: 0': 'index', 'value': 'displacement'}, inplace=True)

# adjust datatypes
tmg_raw = tmg_raw.astype({'group': 'string', 'experiment': 'string', 'measurement': 'string'})

# add time
millisekenden_pro_datenpunkt = 1
tmg_raw['time_ms'] = tmg_raw.groupby(['id', 'experiment', 'measurement'])['index'].transform(lambda x: (x - x.min()) * millisekenden_pro_datenpunkt)

tmg_raw

,index,id,group,experiment,measurement,displacement,time_ms
0,0,15073,beta_ala,posttest,supine_1,0.00,0
1,1,15073,beta_ala,posttest,supine_1,-0.00,1
2,2,15073,beta_ala,posttest,supine_1,-0.00,2
3,3,15073,beta_ala,posttest,supine_1,-0.00,3
4,4,15073,beta_ala,posttest,supine_1,-0.00,4
...,...,...,...,...,...,...,...
500995,501995,93672,beta_ala,pretest,supine_4,0.01,995
500996,501996,93672,beta_ala,pretest,supine_4,0.00,996
500997,501997,93672,beta_ala,pretest,supine_4,0.00,997
500998,501998,93672,beta_ala,pretest,supine_4,0.00,998


In [57]:
example_data = tmg_raw.query('id == 25778 and experiment == "pretest" and measurement == "s4"')

# threshold for peaks Peaks
threshold = 0.04

# Wert des ersten Peaks der Kurve für dieses spezifische Beispiel
peaks, _ = find_peaks(example_data['displacement'].values, height=threshold)
dm = example_data['displacement'].iloc[peaks[0]] if len(peaks) > 0 else np.nan
time_of_dm = example_data['time_ms'].iloc[peaks[0]] if len(peaks) > 0 else np.nan

# Td berechnen: Schwellenwert für Td
threshold_td = 0.1 * dm
# Index von Td berechnen
index_td = example_data.index[example_data['displacement'] >= threshold_td].min()
# Zeit für Td berechnen
time_td = example_data.loc[index_td, 'time_ms'] if pd.notna(index_td) else np.nan

# Schwellenwert für Tc
threshold_tc = 0.9 * dm
# Index von Td berechnen
index_tc = example_data.index[example_data['displacement'] >= threshold_tc].min()
# Zeit für Td berechnen
time_tc = example_data.loc[index_tc, 'time_ms'] if pd.notna(index_tc) else np.nan


# Finde den ersten Wert, der 50% von dm entspricht
threshold_ts = 0.5 * dm
erster_wert_index = None

for i, wert in enumerate(example_data['displacement'].values):
    if wert >= threshold_ts:
        erster_wert_index = i
        break

# Iteriere weiter ab diesem Wert und finde den nächsten Wert, der 50% von dm entspricht
zweiter_wert_index = None

if erster_wert_index is not None:
    for y, wert in enumerate(example_data['displacement'].values[erster_wert_index + 1:]):
        if wert <= threshold_ts:
            zweiter_wert_index = erster_wert_index + 1 + y
            break

# Überprüfe, ob die Indizes gültig sind
if erster_wert_index is not None and zweiter_wert_index is not None:
    erster_wert_index += example_data.index[0]
    zweiter_wert_index += example_data.index[0]

# Berechne die Zeiten für ts_start und ts_end
time_ts_start = example_data.loc[erster_wert_index, 'time_ms'] if erster_wert_index is not None else np.nan
time_ts_end = example_data.loc[zweiter_wert_index, 'time_ms'] if zweiter_wert_index is not None else np.nan

# Finde den ersten Wert, der 90% von dm entspricht
threshold_tr = 0.9 * dm
erste_zahl_index = None

for i, zahl in enumerate(example_data['displacement'].values):
    if zahl >= threshold_tr:
        erste_zahl_index = i
        break

# Iteriere weiter ab diesem Wert und finde den nächsten Wert, der 90% von dm entspricht
zweite_zahl_index = None

if erste_zahl_index is not None:
    for y, zahl in enumerate(example_data['displacement'].values[erste_zahl_index + 1:]):
        if zahl <= threshold_tr:
            zweite_zahl_index = erste_zahl_index + 1 + y
            break

# Überprüfe, ob die Indizes gültig sind
if zweite_zahl_index is not None and zweite_zahl_index is not None:
    erste_zahl_index += example_data.index[0]
    zweite_zahl_index += example_data.index[0]
    
index_tr_end = example_data.index[example_data['time_ms'] == time_ts_end].min()
    
time_tr_start = example_data.loc[zweite_zahl_index, 'time_ms'] if zweite_zahl_index is not None else np.nan
time_tr_end = example_data.loc[index_tr_end, 'time_ms'] if index_tr_end is not None else np.nan

print("Dm:", dm, 'mm')
print("Td:", time_td, 'ms')
print("Tc:", time_tc, 'ms')
print("Ts:", time_ts_end - time_ts_start, 'ms')
print('Tr:', time_tr_end - time_tr_start, 'ms')
print('Vc:', (dm / (time_td + time_tc)), 'mm/ms')


# Deine ggplot-Grafik bleibt unverändert, füge einfach die Punkte für ts_start und ts_end hinzu
(
    ggplot(example_data, aes(x='time_ms', y='displacement'))
    + geom_line()
    + geom_point(data=example_data.iloc[[peaks[0]]], mapping=aes(x='time_ms', y='displacement'), color='red', size=3)
    + geom_point(aes(x='time_tc', y='displacement'), data=pd.DataFrame({'time_tc': [time_tc], 'displacement': [dm * 0.9]}), color='green', size=3)
    + geom_point(aes(x='time_td', y='displacement'), data=pd.DataFrame({'time_td': [time_td], 'displacement': [dm * 0.1]}), color='blue', size=3)
    + geom_point(aes(x='time_ts_start', y='displacement'), data=pd.DataFrame({'time_ts_start': [time_ts_start], 'displacement': [dm * 0.5]}), color='orange', size=3)
    + geom_point(aes(x='time_ts_end', y='displacement'), data=pd.DataFrame({'time_ts_end': [time_ts_end], 'displacement': [dm * 0.5]}), color='orange', size=3)
    + geom_point(aes(x='time_tr_start', y='displacement'), data=pd.DataFrame({'time_tr_start': [time_tr_start], 'displacement': [dm * 0.9]}), color='black', size=3)
    + geom_point(aes(x='time_tr_end', y='displacement'), data=pd.DataFrame({'time_tr_end': [time_tr_end], 'displacement': [dm * 0.9]}), color='black', size=3)
)


Dm: 0.415878055549873 mm
Td: 8 ms
Tc: 29 ms
Ts: 22 ms
Tr: 5 ms
Vc: 0.011239947447293865 mm/ms


# Calculte individual tmg parameters for each measurement

In [58]:
# create dataframe
tmg_parameter = pd.DataFrame(columns=['id', 'group', 'experiment', 'measurement', 'Dm', 'Td', 'Tc', 'Ts', 'Tr', 'Vc'])

# threshold for Peaks
threshold = 0.04

# iterate by ID
for unique_id in tmg_raw['id'].unique():
    # for pre and post test
    for unique_experiment in tmg_raw['experiment'].unique():
        # measurement time point
        for unique_measurement in tmg_raw['measurement'].unique():
            # filter data
            subset = tmg_raw[(tmg_raw['id'] == unique_id) & 
                             (tmg_raw['experiment'] == unique_experiment) & 
                             (tmg_raw['measurement'] == unique_measurement)].copy()
            
            if not subset.empty:
                parameter_value = subset['displacement'].mean()
                
                # group
                group = subset['group'].iloc[0]

                # value of first peak
                peaks, _ = find_peaks(subset['displacement'].values, height=threshold)
                dm = subset['displacement'].iloc[peaks[0]] if len(peaks) > 0 else np.nan

                # Td threshold
                threshold_td = 0.1 * dm
                index_td = subset.index[subset['displacement'] >= threshold_td].min()
                # Td time
                time_td = subset.loc[index_td, 'time_ms'] if pd.notna(index_td) and pd.notna(subset.loc[index_td, 'time_ms']) else np.nan

                # threshold Tc
                threshold_tc = 0.9 * dm
                index_tc = subset.index[subset['displacement'] >= threshold_tc].min()
                # time tc
                time_tc = subset.loc[index_tc, 'time_ms'] if pd.notna(index_tc) and pd.notna(subset.loc[index_tc, 'time_ms']) else np.nan

                # 50% dm
                threshold_ts = 0.5 * dm
                erster_wert_index = None

                for i, wert in enumerate(subset['displacement'].values):
                    if wert >= threshold_ts:
                        erster_wert_index = i
                        break

                # next 50% dm
                zweiter_wert_index = None

                if erster_wert_index is not None:
                    for y, wert in enumerate(subset['displacement'].values[erster_wert_index + 1:]):
                        if wert <= threshold_ts:
                            zweiter_wert_index = erster_wert_index + 1 + y
                            break

                if erster_wert_index is not None and zweiter_wert_index is not None:
                    erster_wert_index += subset.index[0]
                    zweiter_wert_index += subset.index[0]

                # time ts start and end
                time_ts_start = subset.loc[erster_wert_index, 'time_ms'] if pd.notna(erster_wert_index) and pd.notna(subset.loc[erster_wert_index, 'time_ms']) else np.nan
                time_ts_end = subset.loc[zweiter_wert_index, 'time_ms'] if pd.notna(zweiter_wert_index) and pd.notna(subset.loc[zweiter_wert_index, 'time_ms']) else np.nan

                # 90% dm
                threshold_tr = 0.9 * dm
                erste_zahl_index = None

                for i, zahl in enumerate(subset['displacement'].values):
                    if zahl >= threshold_tr:
                        erste_zahl_index = i
                        break

                # next 90% dm
                zweite_zahl_index = None

                if erste_zahl_index is not None:
                    for y, zahl in enumerate(subset['displacement'].values[erste_zahl_index + 1:]):
                        if zahl <= threshold_tr:
                            zweite_zahl_index = erste_zahl_index + 1 + y
                            break

                if zweite_zahl_index is not None and zweite_zahl_index is not None:
                    erste_zahl_index += subset.index[0]
                    zweite_zahl_index += subset.index[0]

                index_tr_end = subset.index[subset['time_ms'] == time_ts_end].min()

                # time tr start and end
                time_tr_start = subset.loc[zweite_zahl_index, 'time_ms'] if pd.notna(zweite_zahl_index) and pd.notna(subset.loc[zweite_zahl_index, 'time_ms']) else np.nan
                time_tr_end = subset.loc[index_tr_end, 'time_ms'] if pd.notna(index_tr_end) and pd.notna(subset.loc[index_tr_end, 'time_ms']) else np.nan

                tmg_parameter = pd.concat([tmg_parameter, pd.DataFrame({
                    'id': [unique_id],
                    'group':[group],
                    'experiment': [unique_experiment],
                    'measurement': [unique_measurement],
                    'Dm': [dm],
                    'Td': [time_td],
                    'Tc': [time_tc],
                    'Ts': [time_ts_end - time_ts_start],
                    'Tr': [time_tr_end - time_tr_start],
                    'Vc': [dm / (time_td + time_tc)]
                })], ignore_index=True)

tmg_parameter


C:\Users\bvenn\AppData\Local\Temp\ipykernel_11308\1770051188.py:94: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tmg_parameter = pd.concat([tmg_parameter, pd.DataFrame({


,id,group,experiment,measurement,Dm,Td,Tc,Ts,Tr,Vc
0,15073,beta_ala,posttest,supine_1,9.95,31,56,77,46,0.11
1,15073,beta_ala,posttest,before_warmup,2.99,23,49,179,146,0.04
2,15073,beta_ala,posttest,after_warmup,2.27,20,43,194,36,0.04
3,15073,beta_ala,posttest,after_pt,1.81,19,40,167,138,0.03
4,15073,beta_ala,posttest,s1,0.74,16,33,26,7,0.02
...,...,...,...,...,...,...,...,...,...,...
496,93672,beta_ala,pretest,s1,3.20,22,41,28,8,0.05
497,93672,beta_ala,pretest,s2,2.28,24,43,30,9,0.03
498,93672,beta_ala,pretest,supine_2,8.41,25,51,137,103,0.11
499,93672,beta_ala,pretest,supine_3,11.59,25,54,144,111,0.15


In [59]:
tmg_parameter = tmg_parameter.astype({'group': 'category', 'experiment': 'category', 'measurement': 'category', 
                                       'Td': 'float64', 'Tc': 'float64', 'Dm': 'float64', 'Ts': 'float64', 'Tr': 'float64',
                                      'Vc': 'float64'})

tmg_parameter['experiment'] = pd.Categorical(tmg_parameter['experiment'], categories=['pretest', 'posttest'], ordered=True)

tmg_parameter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   id           501 non-null    object  
 1   group        501 non-null    category
 2   experiment   501 non-null    category
 3   measurement  501 non-null    category
 4   Dm           501 non-null    float64 
 5   Td           501 non-null    float64 
 6   Tc           501 non-null    float64 
 7   Ts           501 non-null    float64 
 8   Tr           501 non-null    float64 
 9   Vc           501 non-null    float64 
dtypes: category(3), float64(6), object(1)
memory usage: 29.9+ KB


In [60]:
# adding pre or posttest
tmg_parameter['measurement'] = tmg_parameter.apply(lambda row: row['measurement']
                                                   + '_pre' if 'pretest' in row['experiment'] else row['measurement'], axis=1)
tmg_parameter['measurement'] = tmg_parameter.apply(lambda row: row['measurement']
                                                   + '_post' if 'posttest' in row['experiment'] else row['measurement'], axis=1)

In [61]:
tmg_parameter.head(-1)

,id,group,experiment,measurement,Dm,Td,Tc,Ts,Tr,Vc
0,15073,beta_ala,posttest,supine_1_post,9.95,31.00,56.00,77.00,46.00,0.11
1,15073,beta_ala,posttest,before_warmup_post,2.99,23.00,49.00,179.00,146.00,0.04
2,15073,beta_ala,posttest,after_warmup_post,2.27,20.00,43.00,194.00,36.00,0.04
3,15073,beta_ala,posttest,after_pt_post,1.81,19.00,40.00,167.00,138.00,0.03
4,15073,beta_ala,posttest,s1_post,0.74,16.00,33.00,26.00,7.00,0.02
...,...,...,...,...,...,...,...,...,...,...
495,93672,beta_ala,pretest,after_pt_pre,5.18,20.00,37.00,27.00,9.00,0.09
496,93672,beta_ala,pretest,s1_pre,3.20,22.00,41.00,28.00,8.00,0.05
497,93672,beta_ala,pretest,s2_pre,2.28,24.00,43.00,30.00,9.00,0.03
498,93672,beta_ala,pretest,supine_2_pre,8.41,25.00,51.00,137.00,103.00,0.11


# Seated TMG Analysis. Differences from pre to post test

In [62]:
diff_df = pd.read_csv("https://media.githubusercontent.com/media/bvn3141/beta_ala_study/main/tmg_parameter_tmg_diff.csv?token=BGUXDRBYGMAC4JPMSDP4UTLF6AIU4",
                        sep=';')
diff_df = diff_df.astype({'group': 'category', 'experiment': 'category'})
diff_df

,id,group,experiment,Dm,Td,Tc,Ts,Tr,Vc
0,15073,beta_ala,pretest,-1.93,-1,-1,-167,-166,-0.03
1,20438,beta_ala,pretest,-1.33,0,-4,-5,-1,-0.03
2,25778,beta_ala,pretest,-1.23,-9,-4,0,-1,-0.02
3,26107,beta_ala,pretest,-0.98,2,2,-105,-103,-0.02
4,27351,beta_ala,pretest,-0.67,-1,-1,-135,-132,-0.01
5,28514,placebo,pretest,-2.70,-2,-8,-89,-72,-0.04
6,33468,placebo,pretest,1.72,4,28,-55,-140,-0.01
7,39337,placebo,pretest,-1.61,4,6,2,0,-0.03
8,39750,placebo,pretest,-2.10,0,0,-1,-2,-0.04
9,44858,beta_ala,pretest,-0.62,2,4,-63,-63,-0.02


## diff Dm

In [63]:
(
    ggplot(diff_df, aes(x='experiment', y='Dm', color='group', group='group'))
    + geom_boxplot(width=0.5)
    + labs(title='Differences between after warmup and last measurement')
    + ylab('Dm Difference [mm]')
    + xlab('Measurement')
    + scale_x_discrete(limits=['pretest', 'posttest'], labels=['Pre-Test', 'Post-Test'])
)

In [64]:
dm_stat = pg.mixed_anova(dv='Dm',between='group', within='experiment', subject='id', data=diff_df, correction='auto', effsize='np2')
dm_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,0.09,1,22,0.09,0.04,0.84,0.00,NaN
1,experiment,0.05,1,22,0.05,0.08,0.78,0.00,1.00
2,Interaction,0.06,1,22,0.06,0.09,0.76,0.00,NaN


## Diff Td

In [65]:
(
    ggplot(diff_df, aes(x='experiment', y='Td', color='group', group='group'))
    + geom_boxplot(width=0.5)
    + labs(title='Differences between after warmup and last measurement')
    + ylab('Td Difference [ms]')
    + xlab('measurement')
    + scale_x_discrete(limits=['pretest', 'posttest'], labels=['Pre-Test', 'Post-Test'])
)

In [66]:
Td_stat = pg.mixed_anova(dv='Td',between='group', within='experiment', subject='id', data=diff_df, correction='auto', effsize='np2')
Td_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,2.52,1,22,2.52,0.27,0.61,0.01,NaN
1,experiment,4.69,1,22,4.69,1.21,0.28,0.05,1.00
2,Interaction,3.52,1,22,3.52,0.91,0.35,0.04,NaN


## diff Tc

In [67]:
(
    ggplot(diff_df, aes(x='experiment', y='Tc', color='group', group='group'))
    + geom_boxplot(width=0.5)
    + labs(title='Differences between after warmup and last measurement')
    + ylab('Tc Difference [ms]')
    + xlab('measurement')
    + scale_x_discrete(limits=['pretest', 'posttest'], labels=['Pre-Test', 'Post-Test'])
)

In [68]:
Tc_stat = pg.mixed_anova(dv='Tc',between='group', within='experiment', subject='id', data=diff_df, correction='auto', effsize='np2')
Tc_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,17.52,1,22,17.52,0.36,0.56,0.02,NaN
1,experiment,6.02,1,22,6.02,0.28,0.60,0.01,1.00
2,Interaction,11.02,1,22,11.02,0.51,0.48,0.02,NaN


## diff Ts

In [69]:
(
    ggplot(diff_df, aes(x='experiment', y='Ts', color='group', group='group'))
    + geom_boxplot(width=0.5)
    + labs(title='Differences between after warmup and last measurement')
    + ylab('Ts Difference [ms]')
    + xlab('measurement')
    + scale_x_discrete(limits=['pretest', 'posttest'], labels=['Pre-Test', 'Post-Test'])
)

In [70]:
Ts_stat = pg.mixed_anova(dv='Ts',between='group', within='experiment', subject='id', data=diff_df, correction='auto', effsize='np2')
Ts_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,229.69,1,22,229.69,0.02,0.88,0.00,NaN
1,experiment,1485.19,1,22,1485.19,0.95,0.34,0.04,1.00
2,Interaction,105.02,1,22,105.02,0.07,0.80,0.00,NaN


## diff Tr

In [71]:
(
    ggplot(diff_df, aes(x='experiment', y='Tr', color='group', group='group'))
    + geom_boxplot(width=0.5)
    + labs(title='Differences between after warmup and last measurement')
    + ylab('Tr Difference [ms]')
    + xlab('measurement')
    + scale_x_discrete(limits=['pretest', 'posttest'], labels=['Pre-Test', 'Post-Test'])
)

In [72]:
Tr_stat = pg.mixed_anova(dv='Tr',between='group', within='experiment', subject='id', data=diff_df, correction='auto', effsize='np2')
Tr_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,660.08,1,22,660.08,0.06,0.81,0.00,NaN
1,experiment,5808.00,1,22,5808.00,1.75,0.20,0.07,1.00
2,Interaction,133.33,1,22,133.33,0.04,0.84,0.00,NaN


## diff Vc

In [73]:
(
    ggplot(diff_df, aes(x='experiment', y='Vc', color='group', group='group'))
    + geom_boxplot(width=0.5)
    + labs(title='Differences between after warmup and last measurement')
    + ylab('Vc Difference [mm/ms]')
    + xlab('measurement')
    + scale_x_discrete(limits=['pretest', 'posttest'], labels=['Pre-Test', 'Post-Test'])
)

In [74]:
Vc_stat = pg.mixed_anova(dv='Vc',between='group', within='experiment', subject='id', data=diff_df, correction='auto', effsize='np2')
Vc_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,0.00,1,22,0.00,0.00,0.99,0.00,NaN
1,experiment,0.00,1,22,0.00,0.25,0.62,0.01,1.00
2,Interaction,0.00,1,22,0.00,0.20,0.66,0.01,NaN


# Supine TMG Analysis

In [75]:
relevant_measurements = ['supine_1_pre', 'supine_2_pre', 'supine_3_pre', 'supine_4_pre',
                         'supine_1_post', 'supine_2_post', 'supine_3_post', 'supine_4_post']
filtered_data = tmg_parameter[tmg_parameter['measurement'].isin(relevant_measurements)]

filtered_data = filtered_data.sort_values(by=['id', 'group', 'experiment']).reset_index(drop=True)
pretest_data = filtered_data[filtered_data['experiment'] == 'pretest']
posttest_data = filtered_data[filtered_data['experiment'] == 'posttest']


In [76]:
pretest_data = pretest_data.reset_index(drop=True)
pretest_data

,id,group,experiment,measurement,Dm,Td,Tc,Ts,Tr,Vc
0,15073,beta_ala,pretest,supine_1_pre,9.73,31.00,57.00,106.00,72.00,0.11
1,15073,beta_ala,pretest,supine_2_pre,5.77,28.00,56.00,98.00,56.00,0.07
2,15073,beta_ala,pretest,supine_3_pre,9.10,30.00,57.00,149.00,110.00,0.10
3,15073,beta_ala,pretest,supine_4_pre,9.11,29.00,59.00,104.00,64.00,0.10
4,20438,beta_ala,pretest,supine_1_pre,10.23,26.00,50.00,44.00,19.00,0.13
...,...,...,...,...,...,...,...,...,...,...
91,93302,placebo,pretest,supine_4_pre,7.58,27.00,56.00,51.00,21.00,0.09
92,93672,beta_ala,pretest,supine_1_pre,11.23,26.00,53.00,172.00,140.00,0.14
93,93672,beta_ala,pretest,supine_2_pre,8.41,25.00,51.00,137.00,103.00,0.11
94,93672,beta_ala,pretest,supine_3_pre,11.59,25.00,54.00,144.00,111.00,0.15


In [77]:
posttest_data = posttest_data.reset_index(drop=True)
posttest_data

,id,group,experiment,measurement,Dm,Td,Tc,Ts,Tr,Vc
0,15073,beta_ala,posttest,supine_1_post,9.95,31.00,56.00,77.00,46.00,0.11
1,15073,beta_ala,posttest,supine_2_post,6.97,28.00,57.00,86.00,46.00,0.08
2,15073,beta_ala,posttest,supine_3_post,8.98,30.00,57.00,140.00,104.00,0.10
3,15073,beta_ala,posttest,supine_4_post,9.93,30.00,56.00,124.00,88.00,0.12
4,20438,beta_ala,posttest,supine_1_post,8.60,27.00,52.00,49.00,21.00,0.11
...,...,...,...,...,...,...,...,...,...,...
91,93302,placebo,posttest,supine_4_post,8.15,25.00,53.00,49.00,19.00,0.10
92,93672,beta_ala,posttest,supine_1_post,10.28,25.00,49.00,173.00,142.00,0.14
93,93672,beta_ala,posttest,supine_2_post,9.09,25.00,50.00,153.00,123.00,0.12
94,93672,beta_ala,posttest,supine_3_post,11.48,26.00,54.00,136.00,105.00,0.14


In [78]:
result_data = pd.DataFrame()

# Hinzufügen von 'group', 'measurement' und 'id'
result_data['group'] = pretest_data['group']
result_data['measurement'] = pretest_data['measurement']
result_data['id'] = pretest_data['id']

# Berechnung der Differenzen für alle Parameter
diff_columns = ['Dm', 'Td', 'Tc', 'Ts', 'Tr', 'Vc']
for col in diff_columns:
    result_data[f'diff_{col}'] = posttest_data[col] - pretest_data[col]

# Anzeigen des Ergebnisses
result_data

,group,measurement,id,diff_Dm,diff_Td,diff_Tc,diff_Ts,diff_Tr,diff_Vc
0,beta_ala,supine_1_pre,15073,0.22,0.00,-1.00,-29.00,-26.00,0.00
1,beta_ala,supine_2_pre,15073,1.19,0.00,1.00,-12.00,-10.00,0.01
2,beta_ala,supine_3_pre,15073,-0.12,0.00,0.00,-9.00,-6.00,-0.00
3,beta_ala,supine_4_pre,15073,0.81,1.00,-3.00,20.00,24.00,0.01
4,beta_ala,supine_1_pre,20438,-1.63,1.00,2.00,5.00,2.00,-0.03
...,...,...,...,...,...,...,...,...,...
91,placebo,supine_4_pre,93302,0.57,-2.00,-3.00,-2.00,-2.00,0.01
92,beta_ala,supine_1_pre,93672,-0.95,-1.00,-4.00,1.00,2.00,-0.00
93,beta_ala,supine_2_pre,93672,0.68,0.00,-1.00,16.00,20.00,0.01
94,beta_ala,supine_3_pre,93672,-0.12,1.00,0.00,-8.00,-6.00,-0.00


In [79]:
mean_std_supine = result_data.groupby(['group', 'measurement']).agg({
    'diff_Dm': ['mean', 'std'],
    'diff_Td': ['mean', 'std'],
    'diff_Tc': ['mean', 'std'],
    'diff_Ts': ['mean', 'std'],
    'diff_Tr': ['mean', 'std'],
    'diff_Vc': ['mean', 'std'],    
})

mean_std_supine = mean_std_supine.reset_index()
mean_std_supine.columns = ['group', 'measurement',
                           'diff_Dm_mean', 'diff_Dm_std',
                           'diff_Td_mean', 'diff_Td_std',
                           'diff_Tc_mean', 'diff_Tc_std',
                           'diff_Ts_mean', 'diff_Ts_std',
                           'diff_Tr_mean', 'diff_Tr_std',
                           'diff_Vc_mean', 'diff_Vc_std']
mean_std_supine

C:\Users\bvenn\AppData\Local\Temp\ipykernel_11308\1810155329.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_std_supine = result_data.groupby(['group', 'measurement']).agg({


,group,measurement,diff_Dm_mean,diff_Dm_std,diff_Td_mean,diff_Td_std,diff_Tc_mean,diff_Tc_std,diff_Ts_mean,diff_Ts_std,diff_Tr_mean,diff_Tr_std,diff_Vc_mean,diff_Vc_std
0,beta_ala,supine_1_pre,-0.41,1.23,-0.08,1.00,-1.42,3.40,0.58,25.90,2.67,29.37,-0.00,0.02
1,beta_ala,supine_2_pre,0.28,0.70,0.33,2.96,1.17,4.30,5.75,15.90,3.75,16.15,0.00,0.01
2,beta_ala,supine_3_pre,-0.60,1.36,0.00,2.04,-1.17,3.04,6.08,26.01,4.92,28.40,-0.01,0.02
3,beta_ala,supine_4_pre,0.03,0.77,0.25,0.87,-1.42,1.98,-1.58,48.30,-1.17,45.93,0.00,0.01
4,placebo,supine_1_pre,-0.20,1.97,0.00,1.81,-1.58,5.99,-13.75,41.21,-9.33,40.61,0.00,0.03
5,placebo,supine_2_pre,0.36,1.48,-0.58,1.44,-1.42,4.60,-2.58,23.16,-1.42,22.84,0.01,0.02
6,placebo,supine_3_pre,-0.25,1.70,0.00,1.60,0.33,3.28,-19.50,45.36,-12.25,49.90,-0.00,0.02
7,placebo,supine_4_pre,-0.07,1.74,-0.33,1.72,0.33,5.00,-17.00,45.89,-15.75,47.34,-0.00,0.02


In [80]:
# extrahieren des data frames mit differenzen
#result_data.to_excel(r"C:\Users\bvenn\OneDrive\Desktop\Publikation\Dataframe\differences_supine.xlsx")

In [81]:
(
    ggplot(result_data, aes(x='measurement', y='diff_Td', color='group'))
    + geom_boxplot(width=0.5)
    + ylab('Td Difference [ms]')
    + scale_x_discrete(labels=['Supine 1', 'Supine 2', 'Supine 3', 'Supine 4',])
)

In [82]:
# distribution
(
    ggplot(result_data, aes(x='diff_Td'))
    + geom_histogram()
    + ylab('Td Difference [ms]')
)

In [83]:
Td_stat = pg.mixed_anova(dv='diff_Td',between='group', within='measurement', subject='id', data=result_data, correction='auto', effsize='np2')
Td_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,3.01,1,22,3.01,0.62,0.44,0.03,NaN
1,measurement,0.20,3,66,0.07,0.02,0.99,0.00,0.92
2,Interaction,4.11,3,66,1.37,0.52,0.67,0.02,NaN


In [84]:
(
    ggplot(result_data, aes(x='measurement', y='diff_Tc', color='group'))
    + geom_boxplot(width=0.5)
    + ylab('Tc Difference [ms]')
    + scale_x_discrete(labels=['Supine 1', 'Supine 2', 'Supine 3', 'Supine 4',])
)

In [85]:
# distribution
(
    ggplot(result_data, aes(x='diff_Tc'))
    + geom_histogram()
    + ylab('Td Diff [ms]')
)

In [86]:
Tc_stat = pg.mixed_anova(dv='diff_Tc',between='group', within='measurement', subject='id', data=result_data, correction='auto', effsize='np2')
Tc_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,0.38,1,22,0.38,0.02,0.90,0.00,NaN
1,measurement,25.54,3,66,8.51,0.58,0.63,0.03,0.89
2,Interaction,71.71,3,66,23.90,1.63,0.19,0.07,NaN


In [87]:
(
    ggplot(result_data, aes(x='measurement', y='diff_Dm', color='group'))
    + geom_boxplot(width=0.5)
    + ylab('Dm Difference [mm]')
    + scale_x_discrete(labels=['Supine 1', 'Supine 2', 'Supine 3', 'Supine 4',])
)

In [88]:
# distribution
(
    ggplot(result_data, aes(x='diff_Dm'))
    + geom_histogram()
    + ylab('Td Diff [ms]')
)

In [89]:
Dm_stat = pg.mixed_anova(dv='diff_Dm',between='group', within='measurement', subject='id', data=result_data, correction='auto', effsize='np2')
Dm_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,0.44,1,22,0.44,0.10,0.76,0.00,NaN
1,measurement,7.90,3,66,2.63,2.19,0.10,0.09,0.95
2,Interaction,0.70,3,66,0.23,0.19,0.90,0.01,NaN


In [90]:
(
    ggplot(result_data, aes(x='measurement', y='diff_Ts', color='group'))
    + geom_boxplot(width=0.5)
    + ylab('Ts Difference [ms]')
    + scale_x_discrete(labels=['Supine 1', 'Supine 2', 'Supine 3', 'Supine 4',])
)

In [91]:
# distribution
(
    ggplot(result_data, aes(x='diff_Ts'))
    + geom_histogram()
    + ylab('Ts Diff [ms]')
)

In [92]:
Ts_stat = pg.mixed_anova(dv='diff_Ts',between='group', within='measurement', subject='id', data=result_data, correction='auto', effsize='np2')
Ts_stat

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,6080.17,1,22,6080.17,3.04,0.10,0.12,NaN
1,measurement,1606.42,3,66,535.47,0.51,0.68,0.02,0.82
2,Interaction,922.25,3,66,307.42,0.29,0.83,0.01,NaN


In [93]:
(
    ggplot(result_data, aes(x='measurement', y='diff_Tr', color='group'))
    + geom_boxplot(width=0.5)
    + ylab('Tr Difference [ms]')
    + scale_x_discrete(labels=['Supine 1', 'Supine 2', 'Supine 3', 'Supine 4',])
)

In [94]:
# distribution
(
    ggplot(result_data, aes(x='diff_Tr'))
    + geom_histogram()
    + ylab('Tr Diff [ms]')
)

In [95]:
Tr_stat = pg.mixed_anova(dv='diff_Tr',between='group', within='measurement', subject='id', data=result_data, correction='auto', effsize='np2')
Tr_stat

,Source,SS,DF1,DF2,MS,F,p-unc,p-GG-corr,np2,eps,sphericity,W-spher,p-spher
0,group,3589.26,1,22,3589.26,1.68,0.21,NaN,0.07,NaN,NaN,NaN,NaN
1,measurement,1113.53,3,66,371.18,0.33,0.80,0.75,0.01,0.81,False,0.57,0.03
2,Interaction,479.11,3,66,159.70,0.14,0.93,NaN,0.01,NaN,NaN,NaN,NaN


In [96]:
(
    ggplot(result_data, aes(x='measurement', y='diff_Vc', color='group'))
    + geom_boxplot(width=0.5)
    + ylab('Vc Difference [mm/ms]')
    + scale_x_discrete(labels=['Supine 1', 'Supine 2', 'Supine 3', 'Supine 4',])
)

In [97]:
# distribution
(
    ggplot(result_data, aes(x='diff_Vc'))
    + geom_histogram()
    + ylab('Vc Diff [mm/ms]')
)

In [98]:
Vc_stat = pg.mixed_anova(dv='diff_Vc',between='group', within='measurement', subject='id', data=result_data, correction='auto', effsize='np2')
Vc_stat

C:\Users\bvenn\AppData\Roaming\Python\Python311\site-packages\pingouin\distribution.py:1004: RuntimeWarning: divide by zero encountered in scalar divide
  W = np.prod(eig) / (eig.sum() / d) ** d


,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,0.00,1,22,0.00,0.13,0.72,0.01,NaN
1,measurement,0.00,3,66,0.00,1.40,0.25,0.06,0.93
2,Interaction,0.00,3,66,0.00,0.23,0.87,0.01,NaN
